In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, confusion_matrix

In [2]:
df=pd.read_csv('E:\\NAAMII\\Machine_learning\\dataset\\train_set.csv')

In [3]:
df.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238,CLASS
0,ID_1,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,...,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710,0
1,ID_2,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,...,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548,1
2,ID_3,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,...,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704,1
3,ID_4,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,...,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803,0
4,ID_5,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,...,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608,0


In [4]:
df.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238,CLASS
0,ID_1,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,...,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710,0
1,ID_2,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,...,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548,1
2,ID_3,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,...,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704,1
3,ID_4,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,...,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803,0
4,ID_5,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,...,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608,0


In [5]:
y=df['CLASS']
y

0      0
1      1
2      1
3      0
4      0
      ..
310    0
311    1
312    0
313    1
314    0
Name: CLASS, Length: 315, dtype: int64

In [6]:
features = df.drop(['CLASS', 'ID'], axis=1, errors='ignore')


In [7]:
# Check for infinite and extreme values first
print(f"\nData Quality Check:")
inf_count = np.isinf(features.values).sum()
nan_count = np.isnan(features.values).sum()
print(f"Infinite values: {inf_count}")
print(f"NaN values: {nan_count}")


Data Quality Check:
Infinite values: 4
NaN values: 2668


In [8]:
# Replace infinite values with NaN for statistics calculation
features_clean = features.replace([np.inf, -np.inf], np.nan)
features_clean.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,564.936250,...,382.968383,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710
1,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,31.291507,...,452.986164,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548
2,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,11.965643,...,419.781765,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704
3,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,8.966286,...,439.023968,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803
4,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,34.898671,...,485.209184,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608


In [9]:
features_clean.dropna(axis=1, thresh=0.9 * len(features_clean))

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,564.936250,...,382.968383,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710
1,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,31.291507,...,452.986164,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548
2,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,11.965643,...,419.781765,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704
3,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,8.966286,...,439.023968,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803
4,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,34.898671,...,485.209184,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,46787.916667,47002.0,18052.070312,0.385828,0.010883,0.042086,2.874885,0.347840,1.874885,23.499143,...,466.276055,466.276055,6064.0,1.0,585.547823,0.096561,0.0,28.787507,3.894684,0.096561
311,8420.354167,8493.0,4292.039795,0.510004,0.016911,0.101797,2.145061,0.466555,1.145061,25.293867,...,383.044821,383.044821,987.0,1.0,69.155790,0.070369,0.0,29.625473,4.098452,0.070369
312,37262.750000,37407.0,13950.793945,0.374390,0.012759,0.057837,2.585819,0.386725,1.585819,37.474634,...,469.005263,469.005263,4940.0,1.0,505.566802,0.102341,0.0,26.865256,3.815115,0.102341
313,25081.833333,25251.0,11689.275391,0.466045,0.011197,0.044546,2.820962,0.354489,1.820962,18.321132,...,403.597826,403.597826,2944.0,1.0,215.172554,0.073089,0.0,29.180584,4.055504,0.073089


In [10]:
# Impute remaining NaNs with column mean
features_clean.fillna(features_clean.mean(), inplace=True)


In [11]:
# 3. REMOVE ZERO-VARIANCE FEATURES
var_thresh = VarianceThreshold(threshold=0.0)
X_var = var_thresh.fit_transform(features_clean)

In [12]:
# REMOVE HIGHLY CORRELATED FEATURES
def remove_high_correlation_features(X, threshold=0.95):
    corr_matrix = pd.DataFrame(X).corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return pd.DataFrame(X).drop(columns=to_drop, axis=1)

X_clean = remove_high_correlation_features(X_var, threshold=0.95)

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_clean, y)

In [14]:
X_resampled.shape, y_resampled.shape

((382, 721), (382,))

In [15]:
# # 5. SCALING
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_resampled)  # or X_clean if using correlation pruning

# # 6. DIMENSIONALITY REDUCTION WITH PCA (retain 95% variance)
# pca = PCA(n_components=20, random_state=42)
# X_pca = pca.fit_transform(X_scaled)

In [31]:



# Helper function to calculate all metrics
def evaluate(y_true, y_pred, y_proba):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Specificity': specificity,
        'AUROC': roc_auc_score(y_true, y_proba)
    }


# Define models
models = {
    "LogisticRegression": LogisticRegression(class_weight='balanced',max_iter=1000 ,C=1,solver= 'liblinear'),
    "RandomForest": RandomForestClassifier( n_estimators=100, random_state=42,max_depth=10,min_samples_leaf=1, min_samples_split=2),
    "XGBoost": XGBClassifier( eval_metric='logloss', random_state=42,learning_rate=0.1, n_estimators=200, max_depth=3),
    # "SVM":  SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42,C= 0.001),
    "LightGBM": LGBMClassifier(n_estimators= 205,max_depth= 8, learning_rate= 0.04772636524430811,num_leaves= 15,min_child_samples= 15, subsample= 0.8018697037715028,colsample_bytree= 0.6798719929743099,reg_alpha= 0.0027374388764032642,reg_lambda= 0.5886596305304316),
}

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {model_name: [] for model_name in models}

# Loop through models and perform cross-validation
for model_name, model in models.items():
    print(f"\nTraining {model_name}")
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_resampled, y_resampled)):
        X_train, X_val = X_resampled.iloc[train_idx], X_resampled.iloc[val_idx]
        y_train, y_val = y_resampled.iloc[train_idx], y_resampled.iloc[val_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]
        
        metrics = evaluate(y_val, y_pred, y_proba)
        results[model_name].append(metrics)
        print(f" Fold {fold+1}: " + ", ".join([f"{k}: {v:.4f}" for k, v in metrics.items()]))

# Average results
print("\n📊 Average Cross-Validation Results:")
for model_name, folds in results.items():
    avg = {k: np.mean([fold[k] for fold in folds]) for k in folds[0]}
    print(f" {model_name}: " + ", ".join([f"{k}: {v:.4f}" for k, v in avg.items()]))



Training LogisticRegression
 Fold 1: Accuracy: 0.5844, F1: 0.5897, Recall: 0.6053, Specificity: 0.5641, AUROC: 0.6059
 Fold 2: Accuracy: 0.6364, F1: 0.6585, Recall: 0.6923, Specificity: 0.5789, AUROC: 0.6653
 Fold 3: Accuracy: 0.6711, F1: 0.6377, Recall: 0.5789, Specificity: 0.7632, AUROC: 0.6884
 Fold 4: Accuracy: 0.6316, F1: 0.6410, Recall: 0.6579, Specificity: 0.6053, AUROC: 0.7043
 Fold 5: Accuracy: 0.6579, F1: 0.6486, Recall: 0.6316, Specificity: 0.6842, AUROC: 0.6988

Training RandomForest
 Fold 1: Accuracy: 0.6623, F1: 0.6829, Recall: 0.7368, Specificity: 0.5897, AUROC: 0.7483
 Fold 2: Accuracy: 0.6623, F1: 0.6579, Recall: 0.6410, Specificity: 0.6842, AUROC: 0.7240
 Fold 3: Accuracy: 0.7237, F1: 0.7123, Recall: 0.6842, Specificity: 0.7632, AUROC: 0.7438
 Fold 4: Accuracy: 0.7368, F1: 0.7561, Recall: 0.8158, Specificity: 0.6579, AUROC: 0.8047
 Fold 5: Accuracy: 0.6447, F1: 0.6824, Recall: 0.7632, Specificity: 0.5263, AUROC: 0.7195

Training XGBoost
 Fold 1: Accuracy: 0.6753, F1:

In [ ]:
LogisticRegression: Accuracy: 0.6363, F1: 0.6351, Recall: 0.6332, Specificity: 0.6391, AUROC: 0.6725
 RandomForest: Accuracy: 0.6860, F1: 0.6983, Recall: 0.7282, Specificity: 0.6443, AUROC: 0.7481
 XGBoost: Accuracy: 0.6781, F1: 0.6869, Recall: 0.7069, Specificity: 0.6493, AUROC: 0.7372
 LightGBM: Accuracy: 0.6808, F1: 0.6882, Recall: 0.7072, Specificity: 0.6548, AUROC: 0.7103

In [ ]:
# LogisticRegression: Accuracy: 0.5888, F1: 0.6191, Recall: 0.6745, Specificity: 0.5026, AUROC: 0.5946
#  RandomForest: Accuracy: 0.6913, F1: 0.6926, Recall: 0.6966, Specificity: 0.6862, AUROC: 0.7464
#  XGBoost: Accuracy: 0.7043, F1: 0.7162, Recall: 0.7493, Specificity: 0.6596, AUROC: 0.7442
#  SVM: Accuracy: 0.6544, F1: 0.6601, Recall: 0.6750, Specificity: 0.6339, AUROC: 0.6654
#  LightGBM: Accuracy: 0.6602, F1: 0.6701, Recall: 0.6916, Specificity: 0.6285, AUROC: 0.7011
#  NaiveBayes: Accuracy: 0.5889, F1: 0.6667, Recall: 0.8270, Specificity: 0.3502, AUROC: 0.6496

In [ ]:
#  LogisticRegression: Accuracy: 0.5968, F1: 0.5639, Recall: 0.6603, Specificity: 0.5553, AUROC: 0.6514
#  RandomForest: Accuracy: 0.5746, F1: 0.3480, Recall: 0.3060, Specificity: 0.7489, AUROC: 0.5858
#  XGBoost: Accuracy: 0.5968, F1: 0.4617, Recall: 0.4517, Specificity: 0.6911, AUROC: 0.6042
#  SVM: Accuracy: 0.6349, F1: 0.5840, Recall: 0.6447, Specificity: 0.6286, AUROC: 0.6501
#  LightGBM: Accuracy: 0.5778, F1: 0.4734, Recall: 0.4843, Specificity: 0.6387, AUROC: 0.5941
#  NaiveBayes: Accuracy: 0.5778, F1: 0.4417, Recall: 0.4510, Specificity: 0.6596, AUROC: 0.5943

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Build param grid
param_grid = [
    {
        'penalty': ['l1'],
        'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10],
        'solver': ['liblinear'],
        'class_weight': ['balanced']
    },
    {
        'penalty': ['l2'],
        'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10],
        'solver': ['liblinear', 'lbfgs'],
        'class_weight': ['balanced']
    },
    {
        'penalty': ['elasticnet'],
        'C': [0.01, 0.1, 1, 5],
        'solver': ['saga'],
        'l1_ratio': [0.1, 0.5, 0.7, 0.9],
        'class_weight': ['balanced']
    }
]

logreg = LogisticRegression(max_iter=1000, random_state=42)

# Grid search with cross-validation
log_cv = GridSearchCV(
    logreg,
    param_grid,
    scoring='f1',       # Or 'accuracy' if that's your main target
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Fit
log_cv.fit(X_resampled, y_resampled)

# Output best model and params
print("Best parameters:", log_cv.best_params_)
print("Best F1 score:", log_cv.best_score_)


Fitting 5 folds for each of 46 candidates, totalling 230 fits
Best parameters: {'C': 1, 'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}
Best F1 score: 0.7091607405513077


In [20]:
from sklearn.ensemble import RandomForestClassifier

rf_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_cv = GridSearchCV(rf, rf_grid, scoring='f1', cv=5)
rf_cv.fit(X_resampled, y_resampled)

print("Best Random Forest params:", rf_cv.best_params_)

Best Random Forest params: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [21]:
from xgboost import XGBClassifier

xgb_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb = XGBClassifier(scale_pos_weight=191/124, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_cv = GridSearchCV(xgb, xgb_grid, scoring='f1', cv=5)
xgb_cv.fit(X_resampled, y_resampled)

print("Best XGBoost params:", xgb_cv.best_params_)

e:\NAAMII\Machine_learning\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:40:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
e:\NAAMII\Machine_learning\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:41:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
e:\NAAMII\Machine_learning\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:41:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
e:\NAAMII\Machine_learning\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:41:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are

Best XGBoost params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [ ]:
from lightgbm import LGBMClassifier

lgbm_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'class_weight': ['balanced']
}

lgbm = LGBMClassifier(random_state=42)
lgbm_cv = GridSearchCV(lgbm, lgbm_params, scoring='f1', cv=5, n_jobs=-1, verbose=2)
lgbm_cv.fit(X_resampled, y_resampled)
print("Best LGBM params:", lgbm_cv.best_params_)


Fitting 5 folds for each of 5832 candidates, totalling 29160 fits


KeyboardInterrupt: 

In [ ]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, make_scorer

# Define the objective function
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 15, 64),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 30),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'class_weight': 'balanced',
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': -1
    }

    model = LGBMClassifier(**params)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1 = cross_val_score(model, X_resampled, y_resampled, cv=skf, scoring=make_scorer(f1_score)).mean()
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can increase n_trials for more accuracy



e:\NAAMII\Machine_learning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-29 23:23:36,186] A new study created in memory with name: no-name-83c85882-6286-4eb0-bc8f-db4e470aa4a9
[I 2025-05-29 23:23:42,435] Trial 0 finished with value: 0.6803765701372311 and parameters: {'n_estimators': 224, 'max_depth': 4, 'learning_rate': 0.07991495077906864, 'num_leaves': 30, 'min_child_samples': 30, 'subsample': 0.9723642585732775, 'colsample_bytree': 0.6676855322506077, 'reg_alpha': 0.046353388239426696, 'reg_lambda': 0.8106409126188768}. Best is trial 0 with value: 0.6803765701372311.
[I 2025-05-29 23:23:50,343] Trial 1 finished with value: 0.6698974617673805 and parameters: {'n_estimators': 187, 'max_depth': 4, 'learning_rate': 0.024407850951624524, 'num_leaves': 53, 'min_child_samples': 12, 'subsample': 0.92

Best F1 Score: 0.7210126734634894


AttributeError: 'Study' object has no attribute 'best_params_'

In [ ]:
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()


NameError: name 'optuna' is not defined

In [30]:
print("Best F1 Score:", study.best_value)
print("Best Hyperparameters:", study.best_params)

best_model = LGBMClassifier(**study.best_params_, random_state=42)
best_model.fit(X_resampled, y_resampled)


Best F1 Score: 0.7210126734634894
Best Hyperparameters: {'n_estimators': 205, 'max_depth': 8, 'learning_rate': 0.04772636524430811, 'num_leaves': 15, 'min_child_samples': 15, 'subsample': 0.8018697037715028, 'colsample_bytree': 0.6798719929743099, 'reg_alpha': 0.0027374388764032642, 'reg_lambda': 0.5886596305304316}


AttributeError: 'Study' object has no attribute 'best_params_'